In [1]:
import pandas as pd
import numpy as np
import torch
from glob import glob
from torch.utils.data import DataLoader

from ptls.data_load import IterableChain
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.category_size_clip import CategorySizeClip
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.data_load.iterable_processing.target_move import TargetMove
from ptls.data_load.datasets.parquet_dataset import ParquetDataset, ParquetFiles
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter

from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesIterableDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.data_load.datasets import inference_data_loader
from sklearn.model_selection import train_test_split

import pytorch_lightning as pl

import logging
import pickle

from itertools import groupby
from functools import reduce
from operator import iadd

from collections import defaultdict

from ptls.data_load.feature_dict import FeatureDict
from ptls.frames.coles.split_strategy import AbsSplit

from functools import partial
from ptls.nn import TrxEncoder
from ptls.nn.seq_encoder.rnn_encoder import RnnEncoder
from ptls.frames.coles.coles_module import CoLESModule
from ptls.frames.inference_module import InferenceModule

import warnings
warnings.filterwarnings("ignore")

from ptls.data_load.utils import collate_feature_dict
from tqdm.auto import tqdm
import lightgbm as ltb
import json

torch.multiprocessing.set_sharing_strategy('file_system')

# Data preprocessing

In [2]:
train_data_path = 'geo_train_prepr.parquet'
valid_data_path = 'geo_test_prepr.parquet'

In [3]:
process = IterableChain(
            SeqLenFilter(min_seq_len=32),
            ISeqLenLimit(max_seq_len=1024),
            FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
            ToTorch()
)

In [4]:
train = ParquetDataset([train_data_path], post_processing=process, shuffle_files=True)
valid = ParquetDataset([valid_data_path], post_processing=process)

In [5]:
train_dl = DataLoader(
        dataset=train,
        collate_fn=collate_feature_dict,
        shuffle=False,
        num_workers=16,
        batch_size=256,
    )

# Model

In [6]:
from ptls.nn import AggFeatureSeqEncoder

In [7]:
trx_encoder_params = dict(
    was_logified=False,
    log_scale_factor=1,
    numeric_values={},
    embeddings={
        "geohash_4": {'in': 10000},
        "geohash_5": {'in': 10000},
        'geohash_6': {'in': 10000},
      }
)

In [8]:
model = AggFeatureSeqEncoder(**trx_encoder_params)

# Inference

In [9]:
from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset
from datetime import datetime
from ptls.data_load.padded_batch import PaddedBatch

In [10]:
class GetSplit(IterableProcessingDataset):
    def __init__(
        self,
        start_month,
        end_month,
        year=2022,
        col_id='client_id',
        col_time='event_time'
    ):
        super().__init__()
        self.start_month = start_month
        self.end_month = end_month
        self._year = year
        self._col_id = col_id
        self._col_time = col_time
        
    def __iter__(self):
        for rec in self._src:
            for month in range(self.start_month, self.end_month+1):
                features = rec[0] if type(rec) is tuple else rec
                features = features.copy()
                
                if month == 12:
                    month_event_time = datetime(self._year + 1, 1, 1).timestamp()
                else:
                    month_event_time = datetime(self._year, month + 1, 1).timestamp()
                    
                year_event_time = datetime(self._year, 1, 1).timestamp()
                
                mask = features[self._col_time] < month_event_time
                
                for key, tensor in features.items():
                    if key.startswith('target'):
                        features[key] = tensor[month - 1].tolist()    
                    elif key != self._col_id:
                        features[key] = tensor[mask] 
                            
                features[self._col_id] += '_month=' + str(month)

                yield features
                
def collate_feature_dict_with_target(batch, col_id='client_id', targets=False):
    batch_ids = []
    target_cols = []
    for sample in batch:
        batch_ids.append(sample[col_id])
        del sample[col_id]
        
        if targets:
            target_cols.append([sample[f'target_{i}'] for i in range(1, 5)])
            del sample['target_1']
            del sample['target_2']
            del sample['target_3']
            del sample['target_4']
            
    padded_batch = collate_feature_dict(batch)
    if targets:
        return padded_batch, batch_ids, target_cols
    return padded_batch, batch_ids


class InferenceModuleMultimodal(pl.LightningModule):
    def __init__(self, model, pandas_output=True, drop_seq_features=True, model_out_name='out'):
        super().__init__()

        self.model = model
        self.pandas_output = pandas_output
        self.drop_seq_features = drop_seq_features
        self.model_out_name = model_out_name

    def forward(self, x):
        x_len = len(x)
        if x_len == 3:
            x, batch_ids, target_cols = x
        else: 
            x, batch_ids = x
            
        out = self.model(x)
        if x_len == 3:
            target_cols = torch.tensor(target_cols)
            x_out = {
                'client_id': batch_ids,
                'target_1': target_cols[:, 0],
                'target_2': target_cols[:, 1],
                'target_3': target_cols[:, 2],
                'target_4': target_cols[:, 3],
                self.model_out_name: out
            }
        else:
            x_out = {
                'client_id': batch_ids,
                self.model_out_name: out
            }

        if self.pandas_output:
            return self.to_pandas(x_out)
        return x_out

    @staticmethod
    def to_pandas(x):
        expand_cols = []
        scalar_features = {}

        for k, v in x.items():
            if type(v) is torch.Tensor:
                v = v.cpu().numpy()

            if type(v) is list or len(v.shape) == 1:
                scalar_features[k] = v
            elif len(v.shape) == 2:
                expand_cols.append(k)
            else:
                scalar_features[k] = None

        dataframes = [pd.DataFrame(scalar_features)]
        for col in expand_cols:
            v = x[col].cpu().numpy()
            dataframes.append(pd.DataFrame(v, columns=[f'{col}_{i:04d}' for i in range(v.shape[1])]))

        return pd.concat(dataframes, axis=1)

In [11]:
train_process = IterableChain(
            ISeqLenLimit(max_seq_len=1024),
            FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
            GetSplit(start_month=1, end_month=12),
            ToTorch()
)

test_process = IterableChain(
            ISeqLenLimit(max_seq_len=1024),
            FeatureFilter(keep_feature_names=['client_id'], drop_feature_names=['target_1', 'target_2', 'target_3', 'target_4']),
            ToTorch()
)


train = ParquetDataset([train_data_path], post_processing=train_process)
test = ParquetDataset([valid_data_path], post_processing=test_process)

In [12]:
inference_train_dl = DataLoader(
        dataset=train,
        collate_fn=partial(collate_feature_dict_with_target, targets=True),
        shuffle=False,
        num_workers=16,
        batch_size=512,
    )

inference_test_dl = DataLoader(
        dataset=test,
        collate_fn=collate_feature_dict_with_target,
        shuffle=False,
        num_workers=16,
        batch_size=512,
    )

In [13]:
inf_module = InferenceModuleMultimodal(
        model=model,
        pandas_output=True,
        drop_seq_features=True,
        model_out_name='emb',
    )

In [14]:
trainer = pl.Trainer(gpus=[0], max_epochs=-1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
inf_test_embeddings = pd.concat(
        trainer.predict(inf_module, inference_test_dl)
    )
inf_test_embeddings.to_parquet("geo_baseline_agg_test.parquet", index=False, engine="pyarrow", compression="snappy")

In [ ]:
del inf_test_embeddings

In [ ]:
inf_train_embeddings = pd.concat(
        trainer.predict(inf_module, inference_train_dl)
    )
inf_train_embeddings.to_parquet("geo_baseline_agg_train.parquet", index=False, engine="pyarrow", compression="snappy")

In [ ]:
del inf_train_embeddings

# Downstream

In [19]:
class Downstream:
    def __init__(
        self,
        train_path,
        test_path,
        params,
        result_path,
        col_id='client_id',
        targets=(
            'target_1',
            'target_2',
            'target_3',
            'target_4'
        )
    ):
        self.train_path = train_path
        self.test_path = test_path

        self.col_id = col_id
        self.all_targets = targets
        self.params = params
        self.result_path = result_path
        self.drop_feat = list(self.all_targets) + [self.col_id]
        
    def fit(self):
        
        train_embeddings = pd.read_parquet(self.train_path)
        X_train = train_embeddings.drop(columns=self.drop_feat)

        clfs = dict()
        for col_target in self.all_targets:
            clf = ltb.LGBMClassifier(**self.params)
            y_train = train_embeddings[col_target]
            clf.fit(X_train, y_train)
            print(f'Model fitted, target: {col_target}')
            clfs[col_target] = clf
        return clfs

    def get_scores(
        self, 
        clfs
    ):
        scores = pd.DataFrame([])

        test_embeddings_curr = pd.read_parquet(self.test_path).drop_duplicates('client_id')
        X_test = test_embeddings_curr.drop(columns=[self.col_id])
        ids = test_embeddings_curr[self.col_id]
        scores[self.col_id] = ids
            
        for col_target in self.all_targets:
            clf = clfs[col_target]
            score = clf.predict_proba(X_test)[:, 1]
            scores[col_target] = score
        
        return scores

    def run(self):
        clfs = self.fit()
        scores = self.get_scores(clfs)
        
        scores.to_csv(self.result_path)
            
        return scores

In [ ]:
params = {
    "n_estimators": 500,
      "boosting_type": "gbdt",
      "objective": "binary",
      "subsample": 0.5,
      "subsample_freq": 1,
      "learning_rate": 0.02,
      "feature_fraction": 0.75,
      "max_depth": 6,
      "lambda_l1": 1,
      "lambda_l2": 1,
      "min_data_in_leaf": 50,
      "random_state": 42,
      "n_jobs": 8,
}

dw = Downstream(
    train_path="geo_baseline_agg_train.parquet",
    test_path="geo_baseline_agg_test.parquet",
    params=params,
    result_path='baseline_agg_geo.csv' 
)

scores = dw.run()
scores

In [21]:
! python ../scripts/evaluate.py --ref_df_public public_target.parquet --ref_df_private private_target.parquet --pred_df baseline_agg_geo.csv --public_result_path public_geo_agg_score.txt --private_result_path private_geo_agg_score.txt